# S3 Event

In [1]:
context = ''
# Simulate S3 event trigger data
event = {
    "Records": [
        {
            "eventVersion": "2.0",
            "eventTime": "1970-01-01T00:00:00.000Z",
            "requestParameters": {
                "sourceIPAddress": "127.0.0.1"
             },
            "s3": {
                "configurationId": "testConfigRule",
                "object": {
                    "eTag": "0123456789abcdef0123456789abcdef",
                    "sequencer": "0A1B2C3D4E5F678901",
                    "key": "training_input/datasets.h5",
                    "size": 1024
                },
                "bucket": {
                    "arn": "arn:aws:s3:::lnn",
                    "name": "lnn",
                    "ownerIdentity": {
                        "principalId": "EXAMPLE"
                    }
                },
                "s3SchemaVersion": "1.0"
            },
            "responseElements": {
                "x-amz-id-2": "EXAMPLE123/5678abcdefghijklambdaisawesome/mnopqrstuvwxyzABCDEFGH",
                "x-amz-request-id": "EXAMPLE123456789"
            },
            "awsRegion": "us-west-2",
            "eventName": "ObjectCreated:Put",
            "userIdentity": {
                "principalId": "EXAMPLE"
            },
            "eventSource": "aws:s3"
        }
    ]
}

---
# Launch

In [2]:
import launch
from launch import *
launch.lambda_handler(event, context)

Complete Neural Network Settings: 

{
    "activations": {
        "layer1": "sigmoid",
        "layer2": "sigmoid"
    },
    "bias": 0,
    "data_dimensions": {
        "test_set_x": [
            12288,
            50
        ],
        "test_set_y": [
            1,
            50
        ],
        "train_set_x": [
            12288,
            209
        ],
        "train_set_y": [
            1,
            209
        ]
    },
    "data_keys": {
        "bias": "bias|int",
        "m": "m|int",
        "test_set_x": "test_set_x|float64#12288#50",
        "test_set_y": "test_set_y|int64#1#50",
        "train_set_x": "train_set_x|float64#12288#209",
        "train_set_y": "train_set_y|int64#1#209",
        "weights": "weights|float64#12288#1"
    },
    "epoch": 1,
    "epochs": 1,
    "input_data": [
        "train_set_x",
        "train_set_y",
        "test_set_x",
        "test_set_y"
    ],
    "layer": 1,
    "layers": 2,
    "learning_rate": 0.005,
    "neurons": {
     

---
# Launch -> Trainer

In [3]:
event = {
    "parameter_key": "parameters|json",
    "state": "start"
}

In [4]:
import trainer
from trainer import *
trainer.lambda_handler(event, context)

Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "False",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 2,
    "last": "False",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}
Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 3,
    "last": "True",
    "layer": 1,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
# Trainer -> Neuron (Layer 1)

In [5]:
import neuron
from neuron import *

event = {
    "1": {
        "activation": "sigmoid",
        "epoch": 0,
        "id": 1,
        "last": "False",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    },
    "2": {
        "activation": "sigmoid",
        "epoch": 0,
        "id": 2,
        "last": "False",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    },
    "3": {
        "activation": "sigmoid",
        "epoch": 0,
        "id": 3,
        "last": "True",
        "layer": 1,
        "parameter_key": "parameters|json",
        "state": "forward"
    }
}
event = dumps(event)
event = loads(event)

In [6]:
parameter_key = event.get('1')['parameter_key']
parameters = from_cache(endpoint=endpoint, key=parameter_key)
parameters

{'activations': {'layer1': 'sigmoid', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_dimensions': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'data_keys': {'bias': 'bias|int',
  'm': 'm|int',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209',
  'weights': 'weights|float64#12288#1'},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 1,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0}

In [7]:
layer = event.get('1')['layer']
event1 = event.get('1')
neuron.lambda_handler(event=event1, context='')
event2 = event.get('2')
neuron.lambda_handler(event=event2, context='')
event3 = event.get('3')
neuron.lambda_handler(event=event3, context='')

Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
# Neuron (Layer 1) -> Trainer

In [8]:
import trainer
from trainer import *
event = {
    "epoch": 0,
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer.lambda_handler(event, context)

Payload to be sent NeuronLambda: 
{
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}


**Sanity Check: Confirm that `TrainerLambda` processed Layer 1 Activations**

In [9]:
parameters = from_cache(endpoint=endpoint, key='parameters|json')
parameters

{'activations': {'layer1': 'sigmoid', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_dimensions': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'data_keys': {'A1': 'A1|float64#3#209',
  'bias': 'bias|int',
  'm': 'm|int',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209',
  'weights': 'weights|float64#12288#1'},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 2,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0}

In [10]:
A1_key = parameters['data_keys']['A1']
A1 = from_cache(endpoint=endpoint, key=A1_key)
A1.shape

(3, 209)

---
# Trainer -> Neuron (Layer 2)

In [11]:
import neuron
from neuron import *
event = {
    "activation": "sigmoid",
    "epoch": 0,
    "id": 1,
    "last": "True",
    "layer": 2,
    "parameter_key": "parameters|json",
    "state": "forward"
}
neuron.lambda_handler(event, context)

Payload to be sent to TrainerLambda: 
{
    "epoch": 0,
    "layer": 3,
    "parameter_key": "parameters|json",
    "state": "forward"
}


---
# Neuron (Layer 2) -> Trainer

In [12]:
import trainer
from trainer import *
event = {
    "epoch": 0,
    "layer": 3,
    "parameter_key": "parameters|json",
    "state": "forward"
}
trainer.lambda_handler(event, context)

UnboundLocalError: local variable 'results' referenced before assignment

In [13]:
parameters = from_cache(endpoint=endpoint, key="parameters|json")
parameters

{'activations': {'layer1': 'sigmoid', 'layer2': 'sigmoid'},
 'bias': 0,
 'data_dimensions': {'test_set_x': [12288, 50],
  'test_set_y': [1, 50],
  'train_set_x': [12288, 209],
  'train_set_y': [1, 209]},
 'data_keys': {'A1': 'A1|float64#3#209',
  'A2': 'A2|float64#1#209',
  'bias': 'bias|int',
  'm': 'm|int',
  'results': 'results|json',
  'test_set_x': 'test_set_x|float64#12288#50',
  'test_set_y': 'test_set_y|int64#1#50',
  'train_set_x': 'train_set_x|float64#12288#209',
  'train_set_y': 'train_set_y|int64#1#209',
  'weights': 'weights|float64#12288#1'},
 'epoch': 0,
 'epochs': 1,
 'input_data': ['train_set_x', 'train_set_y', 'test_set_x', 'test_set_y'],
 'layer': 3,
 'layers': 2,
 'learning_rate': 0.005,
 'neurons': {'layer1': 3, 'layer2': 1},
 'weight': 0}

# Results: 10/28
This is a good palce to stop. The `TrainerLambda` has updated Layer 2 activations, but failes with processing the Cost and updating the result.